In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.4 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import kerastuner 
import matplotlib.pyplot as plt
import numpy as np 


<ipython-input-2-b9c6421e4879>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner


In [3]:
tf.__version__

'2.12.0'

In [4]:
(x_train ,y_train ) , ( x_test , y_test) = tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [33]:
from tensorflow.python.distribute.distribute_lib import values
def create_model(hp= None):

  num_hidden_layers = 1
  num_units = 8
  dropout_rate = 0.1
  learning_rate = 0.01
  if hp:
    num_hidden_layers = hp.Choice('num_of_hidden_layers', values=[1,2,3])
    num_units = hp.Choice('Neurons' , values = [8 ,16 , 32])
    dropout_rate = hp.Float('dropout_rate', min_value = 0.1 , max_value = 0.5)
    learning_rate =hp.Float('learning rate', min_value=0.0001 , max_value = 0.01)

  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape = (28,28)))
  model.add(tf.keras.layers.Lambda(lambda x : x/255))

  for _ in range(0 , num_hidden_layers):
    model.add(tf.keras.layers.Dense(num_units, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
  model.add(tf.keras.layers.Dense(10 , activation = 'softmax'))
  model.compile(
      loss = 'sparse_categorical_crossentropy',
      optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
      metrics = ['accuracy']

   )
  return model

In [34]:
create_model().summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 lambda_2 (Lambda)           (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 8)                 6280      
                                                                 
 dropout_4 (Dropout)         (None, 8)                 0         
                                                                 
 dense_7 (Dense)             (None, 10)                90        
                                                                 
Total params: 6,370
Trainable params: 6,370
Non-trainable params: 0
_________________________________________________________________


In [40]:
class CustomTuner(kerastuner.tuners.BayesianOptimization):
  def run_trial(self , trial , *args , **kwargs):
    kwargs['batch_size'] = trial.hyperparameters.Int('batch size' , 32 , 128 , step = 32)
    return super(CustomTuner , self).run_trial(trial , *args , **kwargs)

In [41]:
tuner = CustomTuner( 
    create_model ,
    objective = 'val_accuracy',
    max_trials = 20 , 
    directory = 'logs',
    project_name = 'fashion',
    overwrite = True
    )

In [42]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_of_hidden_layers (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
Neurons (Choice)
{'default': 8, 'conditions': [], 'values': [8, 16, 32], 'ordered': True}
dropout_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': None, 'sampling': 'linear'}
learning rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'linear'}


In [47]:
tuner.search(
    x_train , y_train,
    validation_data = (x_test , y_test),
    epochs = 5 , verbose = True 
    
)

In [48]:
tuner.results_summary(1)

Results summary
Results in logs/fashion
Showing 1 best trials
Objective(name="val_accuracy", direction="max")

Trial 15 summary
Hyperparameters:
num_of_hidden_layers: 1
Neurons: 32
dropout_rate: 0.1
learning rate: 0.0047175229630373825
batch size: 32
Score: 0.8359000086784363


In [49]:
bm = tuner.get_best_models(num_models=1)[0]
bm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 lambda (Lambda)             (None, 784)               0         
                                                                 
 dense (Dense)               (None, 32)                25120     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________
